In [80]:
import json
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import haversine_distances
from math import radians
EARTH_RADIUS_M = 6371000
MICRO_CONVERSION = 1e6
US_TO_MS = 1 / 1e3
M_SEC_TO_KM_H = 3.6

In [63]:
with open('../avdata.json') as f:
    d = json.load(f)
print(d.keys())

dict_keys(['acceleration_x', 'acceleration_y', 'acceleration_z', 'accelerator_pedal', 'accelerator_pedal_gradient_sign', 'angular_velocity_omega_x', 'angular_velocity_omega_y', 'angular_velocity_omega_z', 'brake_pressure', 'distance_pulse_front_left', 'distance_pulse_front_right', 'distance_pulse_rear_left', 'distance_pulse_rear_right', 'latitude_degree', 'latitude_direction', 'longitude_degree', 'longitude_direction', 'pitch_angle', 'roll_angle', 'steering_angle_calculated', 'steering_angle_calculated_sign', 'vehicle_speed'])


In [64]:
# Note: timestamps are in microseconds according to https://www.a2d2.audi/a2d2/en/tutorial.html
print(d['vehicle_speed']['unit'])
vals = d['vehicle_speed']['values'] # arrays formatted as [<timestamp>, [velocity]] 

timestamps = np.array(vals)[:, 0]
velocities = np.array(vals)[:, 1]

delta_t = np.append(np.array([0]), timestamps[1:] - timestamps[:-1]) / 1e6

print(timestamps)
print(delta_t)
print(velocities[:100])


Unit_KiloMeterPerHour
[1.53390641e+15 1.53390641e+15 1.53390641e+15 ... 1.53390694e+15
 1.53390694e+15 1.53390694e+15]
[0.       0.019997 0.020005 ... 0.02     0.020001 0.02    ]
[10.36 10.66 10.98 11.28 11.58 11.9  12.18 12.5  12.81 13.13 13.46 13.73
 14.06 14.37 14.69 14.96 15.28 15.56 15.88 16.18 16.44 16.73 17.03 17.32
 17.59 17.87 18.15 18.43 18.73 19.02 19.29 19.58 19.85 20.15 20.43 20.73
 21.03 21.31 21.58 21.83 22.1  22.35 22.61 22.85 23.07 23.3  23.53 23.75
 23.95 24.14 24.34 24.53 24.73 24.9  25.09 25.27 25.43 25.56 25.75 26.02
 26.2  26.31 26.47 26.7  26.89 27.08 27.27 27.43 27.64 27.88 28.08 28.26
 28.47 28.66 28.85 29.01 29.21 29.43 29.6  29.77 29.94 30.09 30.28 30.46
 30.68 30.87 31.06 31.25 31.44 31.62 31.81 31.99 32.2  32.36 32.53 32.7
 32.9  33.06 33.22 33.36]


In [40]:
print(len(timestamps))
print((max(timestamps) - min(timestamps)) / MICRO_CONVERSION)

26288
525.742374


In [38]:
lat = d['latitude_degree']['values']
pt_t = np.array(lat)[:, 0] # get timestamps
lat = np.array(lat)[:, 1]

lng = d['longitude_degree']['values']
lng = np.array(lng)[:, 1]


pts = []
for i in range(len(lat)):
    la, lg = lat[i], lng[i]
    pts.append([radians(la), radians(lg)])

distance_matrix = haversine_distances(pts)
distances = [distance_matrix[i, i - 1] for i in range(1, len(distance_matrix))]
distances = np.append(np.array([0]), distances) * EARTH_RADIUS_M

In [37]:
last_i = 0
for i, delta_d in enumerate(distances):
    if delta_d != 0:
        delta_t = (pt_t[i] - pt_t[last_i]) / MICRO_CONVERSION
        last_i = i
        print(f'distance travelled = {delta_d}, time = {delta_t}, m/s = {delta_d / delta_t}')


8434550466420698
distance travelled = 3.087205742196023, time = 0.800011, m/s = 3.858954117125918
distance travelled = 4.114827177959648, time = 1.120481, m/s = 3.6723756832642835
distance travelled = 7.013847456173301, time = 1.119071, m/s = 6.267562519423076
distance travelled = 9.257780623079357, time = 0.960024, m/s = 9.643280400364322
distance travelled = 10.52882132389683, time = 1.12002, m/s = 9.400565457667568
distance travelled = 11.240130306452572, time = 0.960024, m/s = 11.70817636481231
distance travelled = 11.720813901540375, time = 0.960038, m/s = 12.208697886479884
distance travelled = 11.70546245866958, time = 0.960007, m/s = 12.193101153084903
distance travelled = 12.90470604787036, time = 1.120022, m/s = 11.52183264960006
distance travelled = 11.720810524458935, time = 0.960021, m/s = 12.20891055972623
distance travelled = 11.736596831681458, time = 0.960027, m/s = 12.22527786372827
distance travelled = 11.337115023189568, time = 0.960475, m/s = 11.8036544659565
dista

In [41]:
print((max(pt_t) - min(pt_t)) / MICRO_CONVERSION)

525.602415


In [52]:
def find_nearest(array, value):
    return np.abs(array - value).argmin()
for i, t in enumerate(pt_t):
    idx = find_nearest(timestamps, t)
    print(f'i = {i},\tidx = {idx},\tdiff = {(t - timestamps[idx]) / MICRO_CONVERSION}')

0.003964
i = 2115,	idx = 21152,	diff = 0.003976
i = 2116,	idx = 21160,	diff = 0.003986
i = 2117,	idx = 21168,	diff = 0.003988
i = 2118,	idx = 21176,	diff = 0.003978
i = 2119,	idx = 21192,	diff = 0.003973
i = 2120,	idx = 21200,	diff = 0.003992
i = 2121,	idx = 21208,	diff = 0.003973
i = 2122,	idx = 21216,	diff = 0.003971
i = 2123,	idx = 21232,	diff = 0.003977
i = 2124,	idx = 21240,	diff = 0.003986
i = 2125,	idx = 21248,	diff = 0.003978
i = 2126,	idx = 21256,	diff = 0.003983
i = 2127,	idx = 21272,	diff = 0.003964
i = 2128,	idx = 21280,	diff = 0.003984
i = 2129,	idx = 21288,	diff = 0.00399
i = 2130,	idx = 21296,	diff = 0.003986
i = 2131,	idx = 21312,	diff = 0.003984
i = 2132,	idx = 21320,	diff = 0.003993
i = 2133,	idx = 21328,	diff = 0.004002
i = 2134,	idx = 21336,	diff = 0.003995
i = 2135,	idx = 21352,	diff = 0.004004
i = 2136,	idx = 21360,	diff = 0.004018
i = 2137,	idx = 21368,	diff = 0.004008
i = 2138,	idx = 21376,	diff = 0.004009
i = 2139,	idx = 21392,	diff = 0.004025
i = 2140,	idx = 2

In [70]:
d = {'Distance[m]' : [], 'Vehicle Speed[km/h]' : [], 'Timestamp(ms)' : []}
min_t = min(timestamps)
last_i = 0
for i, delta_d in enumerate(distances):
    if delta_d != 0:
        d_timestamp = pt_t[i]
        idx = find_nearest(timestamps, d_timestamp)
        v_timestamp = timestamps[idx]
        if (v_timestamp - d_timestamp) / MICRO_CONVERSION >= .01:
            print('diff too big')
            assert(False)
        v = velocities[idx]
        d['Distance[m]'].append(delta_d)
        d['Velocity[km/h]'].append(v)
        d['Timestamp(ms)'].append((d_timestamp - min_t) * US_TO_MS)
        



KeyError: 'Velocity[km/h]'

In [71]:
min_t = min(timestamps)
d = {'Vehicle Speed[km/h]' : [], 'Timestamp(ms)' : []}
d['Vehicle Speed[km/h]'] = velocities * 3.6
d['Timestamp(ms)'] = (timestamps - min_t) * US_TO_MS

In [73]:
import utils
df = pd.DataFrame(d)
df['Acceleration[mph/s]'] = utils.get_accel(df)

In [75]:
df['Acceleration[mph/s]'][0] = 0

In [77]:
print(d)

32428.98794484173


In [82]:
def create_trip_csv(d, path):
    vals = d['vehicle_speed']['values'] # arrays formatted as [<timestamp>, [velocity]] 

    timestamps = np.array(vals)[:, 0]
    velocities = np.array(vals)[:, 1]
    min_t = min(timestamps)
    df_dict = {'Vehicle Speed[km/h]' : [], 'Timestamp(ms)' : []}
    df_dict['Vehicle Speed[km/h]'] = velocities * M_SEC_TO_KM_H
    df_dict['Timestamp(ms)'] = (timestamps - min_t) * US_TO_MS

    df = pd.DataFrame(df_dict)
    df['Acceleration[mph/s]'] = utils.get_accel(df)
    df['Acceleration[mph/s]'][0] = 0
    df.to_csv(path)

with open('../avdata.json') as f:
    d = json.load(f)
create_trip_csv(d, '../avdata.csv')